### Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Read in Raw file

In [2]:
# File path
data_route = "../../data/raw_data.csv"
data = pd.read_csv(data_route)
data['Date'] = pd.to_datetime(data['Date'])

In [3]:

daily_data = (
    data.groupby(['Ticker', pd.Grouper(key='Date', freq='D')])  # Group by Ticker and Date (daily frequency)
    .agg({
        'Open': 'first',   # First price of the day
        'High': 'max',     # Highest price of the day
        'Low': 'min',      # Lowest price of the day
        'Close': 'last',   # Last price of the day
        'Volume': 'sum',   # Total volume of the day
        'Ticker': 'size'   # Count the number of hourly data points (transactions)
    })
    .rename(columns={'Ticker': 'Daily_Transactions'})  # Rename the count column
    .reset_index()
)

# Calculate the total number of days of data for each ticker
days_per_ticker = daily_data.groupby('Ticker')['Date'].nunique()

# Display the results
print("Number of days of data for each ticker:")
print(days_per_ticker)

print("\nDaily data with number of transactions:")
print(daily_data.head(10))

# Average daily number of transactions
avg_daily_transactions = daily_data.groupby('Ticker')['Daily_Transactions'].mean()
print("\nAverage daily number of transactions for each ticker:")
print(avg_daily_transactions)


Number of days of data for each ticker:
Ticker
BTC-USD     708
DOGE-USD    708
ETH-USD     708
SOL-USD     708
XRP-USD     708
Name: Date, dtype: int64

Daily data with number of transactions:
    Ticker                      Date          Open          High  \
0  BTC-USD 2023-04-01 00:00:00+00:00  28473.332031  28802.457031   
1  BTC-USD 2023-04-02 00:00:00+00:00  28462.845703  28518.958984   
2  BTC-USD 2023-04-03 00:00:00+00:00  28183.080078  28475.623047   
3  BTC-USD 2023-04-04 00:00:00+00:00  27795.273438  28433.742188   
4  BTC-USD 2023-04-05 00:00:00+00:00  28169.726562  28739.238281   
5  BTC-USD 2023-04-06 00:00:00+00:00  28175.226562  28178.384766   
6  BTC-USD 2023-04-07 00:00:00+00:00  28038.966797  28111.593750   
7  BTC-USD 2023-04-08 00:00:00+00:00  27923.943359  28159.863281   
8  BTC-USD 2023-04-09 00:00:00+00:00  27952.367188  28532.830078   
9  BTC-USD 2023-04-10 00:00:00+00:00  28336.027344  29771.464844   

            Low         Close        Volume  Daily_Transac

This finding makes sense, as SPY does not operate over the weekends.

# Create 3 sets of data: Hourly, 3-hourly, and daily
## Hourly

In [4]:
# Ensure Date is in datetime format and sort data by Ticker and Date
hourly_data = data.copy()
hourly_data['Date'] = pd.to_datetime(hourly_data['Date'])
hourly_data = hourly_data.sort_values(['Ticker', 'Date'])

# Compute **log hourly rv**
hourly_data['ln_hourly_return'] = hourly_data.groupby('Ticker', group_keys=False)['Close'].transform(lambda x: np.log(x).diff())

# Compute **hourly RV** (squared log returns)
hourly_data['hourly_rv'] = hourly_data['ln_hourly_return'] ** 2

# Ln the hourly RV
hourly_data['ln_hourly_rv'] = np.log(hourly_data['hourly_rv'])

# Lag the hourly RV
hourly_data['ln_hourly_rv_lag1'] = hourly_data.groupby('Ticker')['ln_hourly_rv'].shift(1)
hourly_data['ln_hourly_rv_lag2'] = hourly_data.groupby('Ticker')['ln_hourly_rv'].shift(2)
hourly_data['ln_hourly_rv_lag3'] = hourly_data.groupby('Ticker')['ln_hourly_rv'].shift(3)

# Lag RV by 8 and 24 for HAR model
hourly_data['ln_hourly_rv_lag8'] = hourly_data.groupby('Ticker')['ln_hourly_rv'].shift(8)
hourly_data['ln_hourly_rv_lag24'] = hourly_data.groupby('Ticker')['ln_hourly_rv'].shift(24)

## 3 Hourly

In [5]:
# For 3-hourly data, aggregate to non-overlapping 3-hour blocks
three_hourly_data = data.copy()
three_hourly_data['Date'] = pd.to_datetime(three_hourly_data['Date'])
three_hourly_data = three_hourly_data.sort_values(['Ticker', 'Date'])

# Resample: get the last price of each 3-hour window
three_hourly_data = three_hourly_data.groupby(['Ticker', pd.Grouper(key='Date', freq='3H')]).last().reset_index()

# Compute log 3-hourly return (compare current price to price 3 hours ago)
three_hourly_data['ln_3_hourly_return'] = three_hourly_data.groupby('Ticker')['Close'].transform(lambda x: np.log(x).diff())

# For 3-hourly RV, sum the hourly RV over each 3-hour window.
# First, compute hourly RV on raw data.
raw = data.copy()
raw['Date'] = pd.to_datetime(raw['Date'])
raw = raw.sort_values(['Ticker', 'Date'])
raw['ln_hourly_return'] = raw.groupby('Ticker')['Close'].transform(lambda x: np.log(x).diff())
raw['hourly_rv'] = raw['ln_hourly_return'] ** 2

three_hourly_rv = (
    raw.groupby(['Ticker', pd.Grouper(key='Date', freq='3H')])['hourly_rv']
    .sum()
    .reset_index()
    .rename(columns={'hourly_rv': '3_hourly_rv'})
)
# Merge the 3-hourly RV into the three_hourly_data DataFrame
three_hourly_data = three_hourly_data.merge(three_hourly_rv, on=['Ticker', 'Date'], how='left')
three_hourly_data['ln_3_hourly_rv'] = np.log(three_hourly_data['3_hourly_rv']).replace(-np.inf, np.nan)

# Lag the 3-hourly RV
three_hourly_data['ln_3_hourly_rv_lag1'] = three_hourly_data.groupby('Ticker')['ln_3_hourly_rv'].shift(1)
three_hourly_data['ln_3_hourly_rv_lag2'] = three_hourly_data.groupby('Ticker')['ln_3_hourly_rv'].shift(2)
three_hourly_data['ln_3_hourly_rv_lag3'] = three_hourly_data.groupby('Ticker')['ln_3_hourly_rv'].shift(3)

# Lag RV by 4 and 8 for HAR model: This represents a 12 hour and 24 hour lag
three_hourly_data['ln_3_hourly_rv_lag4'] = three_hourly_data.groupby('Ticker')['ln_3_hourly_rv'].shift(4)
three_hourly_data['ln_3_hourly_rv_lag8'] = three_hourly_data.groupby('Ticker')['ln_3_hourly_rv'].shift(8)

C:\Users\young\AppData\Local\Temp\ipykernel_27788\1506711666.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  three_hourly_data = three_hourly_data.groupby(['Ticker', pd.Grouper(key='Date', freq='3H')]).last().reset_index()
C:\Users\young\AppData\Local\Temp\ipykernel_27788\1506711666.py:21: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  raw.groupby(['Ticker', pd.Grouper(key='Date', freq='3H')])['hourly_rv']


## Daily


In [6]:

daily_data = data.copy()
daily_data['Date'] = pd.to_datetime(daily_data['Date'])
daily_data = daily_data.sort_values(['Ticker', 'Date'])

# Resample: get the last price of each day (or use a method that suits your analysis)
daily_data = daily_data.groupby(['Ticker', pd.Grouper(key='Date', freq='D')]).last().reset_index()

# Compute daily RV by summing the hourly RVs
# Recompute hourly RV on raw data if necessary
raw['Date'] = pd.to_datetime(raw['Date'])
raw = raw.sort_values(['Ticker', 'Date'])
raw['ln_hourly_return'] = raw.groupby('Ticker')['Close'].transform(lambda x: np.log(x).diff())
raw['hourly_rv'] = raw['ln_hourly_return'] ** 2

daily_rv = (
    raw.groupby(['Ticker', pd.Grouper(key='Date', freq='D')])['hourly_rv']
    .sum()
    .reset_index()
    .rename(columns={'hourly_rv': 'daily_rv'})
)
daily_data = daily_data.merge(daily_rv, on=['Ticker', 'Date'], how='left')
daily_data['ln_daily_rv'] = np.log(daily_data['daily_rv']).replace(-np.inf, np.nan)

# Compute daily returns from the daily closing prices
daily_data['daily_return'] = daily_data.groupby('Ticker')['Close'].pct_change()
daily_data['ln_daily_return'] = np.log(daily_data['Close'] / daily_data.groupby('Ticker')['Close'].shift(1))

# Lag the daily RV
daily_data['ln_daily_rv_lag1'] = daily_data.groupby('Ticker')['ln_daily_rv'].shift(1)
daily_data['ln_daily_rv_lag2'] = daily_data.groupby('Ticker')['ln_daily_rv'].shift(2)
daily_data['ln_daily_rv_lag3'] = daily_data.groupby('Ticker')['ln_daily_rv'].shift(3)

# Calculate the weekly rv: Sum of past 7 days, and then divide by 7
daily_data['weekly_rv'] = daily_data.groupby('Ticker')['daily_rv'].rolling(window=7).sum().reset_index(0, drop=True) / 7
daily_data['ln_weekly_rv'] = np.log(daily_data['weekly_rv']).replace(-np.inf, np.nan)
daily_data['ln_weekly_rv_lag1'] = daily_data.groupby('Ticker')['ln_weekly_rv'].shift(1)

# Calculate the monthly rv: Sum of past 30 days, and then divide by 30
daily_data['monthly_rv'] = daily_data.groupby('Ticker')['daily_rv'].rolling(window=30).sum().reset_index(0, drop=True) / 30
daily_data['ln_monthly_rv'] = np.log(daily_data['monthly_rv']).replace(-np.inf, np.nan)
daily_data['ln_monthly_rv_lag1'] = daily_data.groupby('Ticker')['ln_monthly_rv'].shift(1)


## Classification
In my model, I wish to classify each coin as either high risk, medium risk, or low risk, based on their hourly realised variance. High RV constitutes as high risk, and likewise for medium risk and low risk. I've used a weighting system to determine the risk level

### Use the training data to do the aggregation

In [7]:
# Make date column datetime
data['Date'] = pd.to_datetime(data['Date'])

# Base the eda on the training data
data = data.sort_values('Date')

# Determine when the first year ends, and use it as train data
# The rest of the data is used as test data
min_date = data['Date'].min()
max_date = data['Date'].max()

# Calculate the total time span of the data
total_time_span = max_date - min_date

# Define the first year of data
first_year_end = min_date + pd.DateOffset(years=1)

# Filter data for the first year
first_year_data = data[data['Date'] <= first_year_end]

# Calculate the percentage of data in the first year
percentage_first_year = (len(first_year_data) / len(data))

train_split = percentage_first_year
train_data = data[:int(train_split * len(data))]
test_data = data[int(train_split * len(data)):]

# Print train and test data date
print(train_data['Date'].min(), train_data['Date'].max())
print(test_data['Date'].min(), test_data['Date'].max())

2023-04-01 00:00:00+00:00 2024-04-01 00:00:00+00:00
2024-04-01 01:00:00+00:00 2025-03-09 23:00:00+00:00


In [8]:
# Filter training data by date (for hourly, 3-hourly, and daily data)
hourly_data_train = hourly_data[hourly_data['Date'] <= first_year_end]
three_hourly_data_train = three_hourly_data[three_hourly_data['Date'] <= first_year_end]
daily_data_train = daily_data[daily_data['Date'] <= first_year_end]

# Compute mean log returns for each ticker
hourly_data_mean = hourly_data_train.groupby('Ticker')['ln_hourly_rv'].mean()
three_hourly_data_mean = three_hourly_data_train.groupby('Ticker')['ln_3_hourly_rv'].mean()
daily_data_mean = daily_data_train.groupby('Ticker')['ln_daily_rv'].mean()

risk_df = pd.concat([hourly_data_mean, three_hourly_data_mean, daily_data_mean], axis=1).reset_index()
risk_df.columns = ['Ticker', 'mean_ln_hourly_rv', 'mean_ln_3_hourly_rv', 'mean_ln_daily_rv']

# Define weights: e.g., giving more weight to the daily return since it's more stable.
weights = {
    'mean_ln_hourly_rv': 0.2,
    'mean_ln_3_hourly_rv': 0.3,
    'mean_ln_daily_rv': 0.5
}

risk_df['Composite_Score'] = (
    risk_df['mean_ln_hourly_rv'] * weights['mean_ln_hourly_rv'] +
    risk_df['mean_ln_3_hourly_rv'] * weights['mean_ln_3_hourly_rv'] +
    risk_df['mean_ln_daily_rv'] * weights['mean_ln_daily_rv']
)

# --- Define Risk Bins and Classify ---
# Create bins using the 33rd and 66th percentiles as thresholds.
bins = [
    risk_df['Composite_Score'].min(),
    risk_df['Composite_Score'].quantile(0.33),
    risk_df['Composite_Score'].quantile(0.66),
    risk_df['Composite_Score'].max()
]
bins = sorted(list(set(bins)))  # Remove duplicates if any and sort

# Define risk labels
risk_labels = ['Low Risk', 'Medium Risk', 'High Risk']

# Classify tickers into risk groups based on the composite score
risk_df['Risk'] = pd.cut(
    risk_df['Composite_Score'],
    bins=bins,
    labels=risk_labels,
    include_lowest=True
)

# Sort
risk_df = risk_df.sort_values('Composite_Score')
print("Risk Classification:")
print(risk_df)
risk_df.to_csv('../../data/risk_classification.csv', index=False)

hourly_data = hourly_data.merge(risk_df[['Ticker', 'Risk']], on='Ticker', how='left')
three_hourly_data = three_hourly_data.merge(risk_df[['Ticker', 'Risk']], on='Ticker', how='left')
daily_data = daily_data.merge(risk_df[['Ticker', 'Risk']], on='Ticker', how='left')

print("3-hourly data with risk classification:")
print(three_hourly_data.columns)


# Save the data
hourly_data.to_csv('../../data/hourly_data.csv', index=False)
three_hourly_data.to_csv('../../data/three_hourly_data.csv', index=False)
daily_data.to_csv('../../data/daily_data.csv', index=False)


Risk Classification:
     Ticker  mean_ln_hourly_rv  mean_ln_3_hourly_rv  mean_ln_daily_rv  \
0   BTC-USD         -12.999954           -10.951066         -8.212067   
2   ETH-USD         -12.789053           -10.688543         -7.948952   
4   XRP-USD         -12.265120           -10.170671         -7.456567   
1  DOGE-USD         -12.126309           -10.025884         -7.292734   
3   SOL-USD         -11.200828            -9.144045         -6.520270   

   Composite_Score         Risk  
0        -9.991344     Low Risk  
2        -9.738849     Low Risk  
4        -9.232509  Medium Risk  
1        -9.079394    High Risk  
3        -8.243514    High Risk  
3-hourly data with risk classification:
Index(['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume',
       'ln_3_hourly_return', '3_hourly_rv', 'ln_3_hourly_rv',
       'ln_3_hourly_rv_lag1', 'ln_3_hourly_rv_lag2', 'ln_3_hourly_rv_lag3',
       'ln_3_hourly_rv_lag4', 'ln_3_hourly_rv_lag8', 'Risk'],
      dtype='object')
